In [ ]:
!pip install datasets
from datasets import load_dataset
sst_dataset=load_dataset("sst")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for sst contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/sst
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


getting training data sentences and labels


In [ ]:
train_data = sst_dataset["train"]
sentences=train_data["sentence"]
scores= train_data["label"]
sentences = [s.lower() for s in sentences]

classify sentences according to score

•From 0 to 0.2 (0.2 included) will be class 0 “very negative”.
•From 0.2 to 0.4 (0.4 included) will be class 1 “negative”.
•From 0.4 to 0.6 (0.6included)will be class 2 “neutral”.•From 0.6to 0.8 (0.8 included)will be class 3 “positive”.•From 0.8to 1.0(1.0included)will be class 4 “very positive”.

In [ ]:
import numpy as np
classes = [[] for _ in range(5)]
#put each sentence in the class that it belongs to
i=0
for score in scores:
    if score <= 0.2:
      scores[i]=0
      classes[0].append(sentences[i])
    elif score > 0.2 and score <= 0.4:
      scores[i]=1
      classes[1].append(sentences[i])
    elif score > 0.4 and score <=0.6:
      scores[i]=2
      classes[2].append(sentences[i])
    elif score > 0.6 and score <=0.8:
      scores[i]=3
      classes[3].append(sentences[i])
    else:
      scores[i]=4
      classes[4].append(sentences[i])
    i+=1

count number of occurences of all words in the document


In [ ]:
from collections import Counter

sentences_np=np.array(sentences)
all_words = ' '.join(sentences_np).split()
word_counts = Counter(all_words)
word_counts_dict = dict(word_counts)
del word_counts
del sentences_np
del all_words
print(word_counts_dict['rock'])
print(len(word_counts_dict))

25
16579


naivee bayes

In [ ]:
def countWord(goal,words):#count words in a single class
    return  np.sum(words == goal)

training loop


In [ ]:
from math import log2
words_probablites={0: {},#will store the likelihood of each word in each class
                  1: {},
                   2: {},
                   3: {},4: {}}
Ndoc = sum(len(row) for row in classes)# counts the total number of words
priors=[]

for k in range(len(classes)):
  Nc=len(classes[k])#count total number of words in each class
  myclass=np.array(classes[k])
  words = ' '.join(myclass).split()
  word_counts = Counter(words)
  this_word_counts_dict = dict(word_counts)#stores words count in each class to measure likelihood
  priors.append(log2(Nc/Ndoc))
  for word in this_word_counts_dict:
      if word not in words_probablites[k] :#and len(word) > 1
        #occurence=countWord(word,words)
        log_likelihood=np.float16(log2((this_word_counts_dict[word]+1)/(word_counts_dict[word]+1)))
        words_probablites[k][word]=log_likelihood
words_probablites[4]['rock']
this_word_counts_dict['rock']




4

test naivee bayes


In [ ]:
final_sum= [[] for _ in range(5)] #will contian the probability of each sentence to be in each class
j=0
for i in range (len(classes)):
  sum=priors[i]
  for sentence in sentences:
    words=sentence.split(' ')
    for word in words:#measures the probabily of each sentence for a specific class
      #if len(word) > 1:#skip one letter words
        if word in words_probablites[i]:#check if word appeared before in this class
          sum+=words_probablites[i][word]
    final_sum[i].append(sum)
    j+=1


max_indices = np.argmax(final_sum, axis=0)
print(len(max_indices))
max_indices[0]




8544


1



```
# This is formatted as code
```

sklearn naivee bayes



In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
model=make_pipeline(CountVectorizer(),MultinomialNB(alpha=1))
model.fit(sentences,scores)
labels = model.predict(sentences)
label[0]

[3, 4, 2, 1, 0]